<a href="https://colab.research.google.com/github/EitMD/EspIDMD/blob/main/churn_ju.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ACME INC.
### *MARKETING*


Mejorando su programa que predice el riesgo de abandono del servicio por parte de sus clientes.

###Lectura del Dataset

Importamos las librerías a utilizar.  
En este caso Pandas


In [4]:
#import pandas as pd

Cargamos el dataset en un dataframe.

In [5]:
#df = pd.read_csv('churn.csv')

Obtenemos la dimensión del dataset

In [6]:
#print('The shape of the dataset is:', df.shape)

Limpieza de datos

Selección de variables

Separación de base de datos

Implementación de algoritmo(s)

Entrenar modelo


validar modelo

Gráficar (validación)

Selección de dataframe para predicción

Ejecutar predicciones